# Lab 3 - Linear Regression


- Your name: Phan Quốc Lưu

- Your student code: 20133065

## I. Hướng dẫn

### 1.1. Khởi tạo Spark

In [115]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = (SparkSession
         .builder
         .appName("Linear Regression")
         .getOrCreate())

### 1.2. Đọc và load tập dữ liệu Boston housing

In [116]:
bostonDF = (spark.read
            .option("HEADER", True)
            .option("inferSchema", True)
            .csv("./data/BostonHousing.csv")
           )

bostonDF.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



### 1.3. Giới thiệu tập dữ liệu Boston housing

`crim`: per capita crime rate by town.

`zn`: proportion of residential land zoned for lots over 25,000 sq.ft.

`indus`: proportion of non-retail business acres per town.

`chas`: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

`nox`: nitrogen oxides concentration (parts per 10 million).

`rm`: average number of rooms per dwelling.

`age`: proportion of owner-occupied units built prior to 1940.

`dis`: weighted mean of distances to five Boston employment centres.

`rad`: index of accessibility to radial highways.

`tax`: full-value property-tax rate per 10,000 dollars.

`ptratio`: pupil-teacher ratio by town.

`b` (`black`): $1000(Bk - 0.63)^2$ where $Bk$ is the proportion of blacks by town.

`lstat`: lower status of the population (percent).

`medv`: median value of owner-occupied homes in 1000 dollars.

### 1.4. Tạo các biến input (features) và output (label)

Spark khác với nhiều machine learning framework khác ở chỗ ta cần huấn luyện mô hình của mình trên một cột duy nhất có chứa một vectơ gồm tất cả các feature (attribute) mà ta quan tâm. Ta sẽ chuẩn bị dữ liệu bằng cách tạo một cột có tên `features` gồm các thuộc tính `rm` (average number of rooms), `crim` (crime rate),  và `lstat` (lower status of the population).

Thêm cột `features` vừa tạo vào data frame sử dụng `VectorAssembler`:

In [117]:
from pyspark.ml.feature import VectorAssembler

featureCols = ["rm", "crim", "lstat"]
assembler = VectorAssembler(inputCols = featureCols, outputCol = "features")
bostonFeaturizedDF = assembler.transform(bostonDF)

bostonFeaturizedDF.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[6.575,0.00632,4.98]|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[6.421,0.02731,9.14]|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[7.185,0.02729,4.03]|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[6.998,0.03237,2.94]|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[7.147,0.06905,5.33]|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 rows



### 1.5. Tạo mô hình Linear Regression

Import thư viện linear regression và thiết lập output là thuộc tính `medv`, input là `features`. Tham khảo thêm về các tham số của thư viện LinearRegression ở [đây](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html#pyspark.ml.regression.LinearRegression).

In [118]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol = "medv", featuresCol = "features")

### 1.6. Fit mô hình với data

In [119]:
lrModel = lr.fit(bostonFeaturizedDF)

### 1.7. Xem các tham số của mô hình sau khi đã fit với data

In [120]:
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))

Coefficients: 5.2, -0.1, -0.6
Intercept: -2.6


**Ý nghĩa**: $predicted\_medv = (5.2 * rm) - (0.1 * crim) - (0.6 * lstat) - 2.6$

hay $predicted\_medv = (5.2 * number\_of\_rooms) - (0.1 * crime\_rate) - (0.6 * lower\_class) - 2.6$

Hai độ đo đánh giá thường được dùng là Root Mean Squared Error (RMSE) và R-Squared score ($R^2$).

$$RMSE(y, \hat{y}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y_i})^2}$$

$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y_i})^2}{\sum_{i=1}^{n} (y_i - \overline{y})^2}$$

Trong đó:

- $\hat{y_i}$ là giá trị mà mô hình dự đoán cho $x_i$
    
- $y_i$ là output thật sự cho $x_i$
    
- $n$ là số lượng phần tử trong tập dữ liệu
    
- $\overline{y} = \frac{1}{n} \sum_{i=1}^{n} y_i$

In [121]:
print("R-Squared score: {}".format(lrModel.summary.r2))

R-Squared score: 0.6458520515781128


**Ý nghĩa**: mô hình có thể giải thích khoảng $65 \%$ sự biến thiên (variance) trong dữ liệu. 

Một số độ đo và chỉ số khác:

In [122]:
lrModelSummary = lrModel.summary

print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

MAE score: 3.8912949765069533
RMSE score: 5.4678160740273904
Coefficient Standard Errors: [0.4420347151349426, 0.032022216026543225, 0.047669471406803186, 3.1660227928472815]
T Values: [11.80213848770088, -3.214670921988156, -12.135352093519293, -0.809296451597215]
P Values: [0.0, 0.0013900026454840564, 0.0, 0.41872805807610836]


Các chỉ số `Standard Errors`, `t-values`, `p-values` liên quan đến ý nghĩa thống kê (statistically significant) của các hệ số (tham số) của mô hình. Xem thêm về diễn giải của các chỉ số này ở [đây](https://dss.princeton.edu/online_help/analysis/interpreting_regression.htm) hoặc [đây](https://boostedml.com/2019/06/linear-regression-in-r-interpreting-summarylm.html).

### 1.8. Sử dụng mô hình

Chọn ra 10 dòng dữ liệu đầu tiên (xem như nó là dữ liệu mới). Ta sẽ xem dự đoán của mô hình trên 10 dòng dữ liệu này và cho sánh nó với giá trị thực tế.

In [123]:
subsetDF = (bostonFeaturizedDF
            .limit(10)
            .select("features", "medv")
           )

subsetDF.show(5)

+--------------------+----+
|            features|medv|
+--------------------+----+
|[6.575,0.00632,4.98]|24.0|
|[6.421,0.02731,9.14]|21.6|
|[7.185,0.02729,4.03]|34.7|
|[6.998,0.03237,2.94]|33.4|
|[7.147,0.06905,5.33]|36.2|
+--------------------+----+
only showing top 5 rows



Sử dụng phương thức `transform` trên mô hình được huấn luyện để xem dự đoán của nó.

`lrModel` là một estimator, ta có thể biến đổi dữ liệu bằng cách sử dụng phương thức `.transform()` của nó.

In [124]:
predictionDF = lrModel.transform(subsetDF)

predictionDF.show(5)

+--------------------+----+------------------+
|            features|medv|        prediction|
+--------------------+----+------------------+
|[6.575,0.00632,4.98]|24.0|28.857717647784423|
|[6.421,0.02731,9.14]|21.6|25.645644850540144|
|[7.185,0.02729,4.03]|34.7| 32.58746300992212|
|[6.998,0.03237,2.94]|33.4| 32.24191904275643|
|[7.147,0.06905,5.33]|36.2| 31.63288834584224|
+--------------------+----+------------------+
only showing top 5 rows



Bây giờ, ta thử dự đoán giá nhà cho một điểm dữ liệu mới của một ngôi nhà 6 phòng ngủ (`rm = 6`), với tỷ lệ tội phạm là 3.6 (`crim = 3.6`), và tầng lớp có thu nhập thấp hơn trung bình là 12% (`lstat = 12`). Theo công thức ở trên, mô hình sẽ dự đoán giá nhà khoảng 21.

In [125]:
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([6., 3.6, 12.]), )]              # Tạo new data point
predictDF = spark.createDataFrame(data, ["features"])

lrModel.transform(predictDF).show()

+--------------+------------------+
|      features|        prediction|
+--------------+------------------+
|[6.0,3.6,12.0]|21.427061506649363|
+--------------+------------------+



### 1.9. Tạo ML pipeline và đánh giá dùng phương pháp hold out

![holdout_model_selection](./image/holdout_model_selection.JPG)

In [126]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler

import pprint

pp = pprint.PrettyPrinter(indent = 4)

# Create a LinearRegression instance. This instance is an Estimator.
lr = LinearRegression(labelCol="medv", featuresCol="features")

# Create a StandardScaler to normalize each feature to have zero mean and unit standard deviation
scaler = StandardScaler(inputCol = "features", outputCol = "scaledFeatures",
                        withStd = True, withMean = True)

# Configure an ML pipeline, which consists of two stages: scaler and lr.
pipeline = Pipeline(stages = [scaler, lr])

# Specify hyper-parameters
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [1, 0.1, 0.01]) \
    .build()

# Train/test split
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed = 1)

# Setup TrainValidationSplit 
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# trainRatio = 0.8: 80% of the data will be used for training, 20% for testing
tvs = TrainValidationSplit(estimator = pipeline, 
                           estimatorParamMaps = paramGrid, 
                           evaluator = RegressionEvaluator(labelCol = "medv", metricName = "rmse"), 
                           trainRatio = 0.8)

# Run TrainValidationSplit on training data, and choose the best set of parameters
lrModel = tvs.fit(trainDF)

# Print rmse on validation set for each `regParam`: 1, 0.1, 0.01
print(lrModel.validationMetrics)

# Make predictions on test data. cvModel uses the best model found (regParam = 0.1)
prediction = lrModel.transform(testDF)
result = prediction.select("features", "scaledFeatures", "medv", "prediction").collect()

# Print some predictions
for row in result[0:5]:
    pp.pprint("features=%s, scaledFeatures=%s, label=%s -> prediction=%s" % 
              (row.features, row.scaledFeatures, row.medv, row.prediction))

[7.7366294084890095, 7.70373787178677, 7.702231388666923]
('features=[7.249,0.01311,4.81], '
 'scaledFeatures=[1.4164431814772305,-0.4648881655422324,-1.092220358196006], '
 'label=35.4 -> prediction=32.31865958601522')
('features=[7.135,0.01778,4.45], '
 'scaledFeatures=[1.2499947891838228,-0.4642127043800687,-1.142381331431606], '
 'label=32.9 -> prediction=31.945823700553785')
('features=[6.516,0.0187,6.36], '
 'scaledFeatures=[0.3462092205029505,-0.46407963708473876,-0.8762495012093932], '
 'label=23.1 -> prediction=27.662350320859627')
('features=[7.104,0.01951,8.05], '
 'scaledFeatures=[1.2047325070689492,-0.4639624800095027,-0.6407715990756024], '
 'label=33.0 -> prediction=29.681160418312835')
('features=[8.034,0.02009,2.88], '
 'scaledFeatures=[2.5626009705151724,-0.463878589758099,-1.36113890915353], '
 'label=50.0 -> prediction=37.46230771138211')


## II. Câu hỏi

### **Câu hỏi 1**: Huấn luyện mô hình Linear Regression và dùng nó để dự đoán cho dữ liệu mới

#### a. Tạo mô hình Linear Regression

Tạo một mô hình linear regression mới với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

i. Tạo biến `newFeatures` cho huấn luyện mô hình

In [127]:
# Write your code here
from pyspark.ml.feature import VectorAssembler

featureCols = ["indus", "age", "dis"] 
assembler = VectorAssembler(inputCols = featureCols, outputCol = "newFeatures")
bostonFeaturizedDF = assembler.transform(bostonDF)

ii. Huấn luyện mô hình

In [128]:
# Write your code here
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="medv", featuresCol="newFeatures")
lrModel = lr.fit(bostonFeaturizedDF)

iii. In kết quả của mô hình (các hệ số và $R^2$ score )

In [129]:
# Write your code here
print("Coefficients: {0:.3f}, {1:.3f}, {2:.3f}".format(*lrModel.coefficients)) 
print("Intercept: {0:.3f}".format(lrModel.intercept))
print("R-squared: {0:.3f}".format(lrModel.summary.r2))


Coefficients: -0.736, -0.094, -1.545
Intercept: 43.034
R-squared: 0.285


iv. Diễn giải các hệ số (`Coefficients`, `Intercept`), $R^2$ `score`, `RMSE score`, các `Standard Errors`, `t-values`, `p-values` của mô hình.

In [130]:
# Write your code here
lrModelSummary = lrModel.summary

print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

MAE score: 5.560969336121602
RMSE score: 7.766715476913002
Coefficient Standard Errors: [0.07389071808161016, 0.019157338599964224, 0.27719285411308525, 2.315290077905648]
T Values: [-9.954031888835738, -4.907552574046968, -5.573530886805173, 18.58698533272279]
P Values: [0.0, 1.2485850908738882e-06, 4.079797522038575e-08, 0.0]


#### b. Sử dụng mô hình

Dùng mô hình huấn luyện được để dự đoán cho các điểm dữ liệu mới có (`indus`, `age`, `dis`) lần lượt là `(11, 68, 4)`, `(6, 35, 2)`, `(19, 74, 8)`.

In [131]:
# Write your code here
assembler = VectorAssembler(inputCols=["indus", "age", "dis"], 
                           outputCol="newFeatures")
data = [(11, 68, 4), (6, 35, 2), (19, 74, 8)]
predictions_df = lrModel.transform(new_data_df)
predictions_df.show()


+-----+---+---+---------------+------------------+
|indus|age|dis|    newFeatures|        prediction|
+-----+---+---+---------------+------------------+
|   11| 68|  4|[11.0,68.0,4.0]|22.370810825866748|
|    6| 35|  2| [6.0,35.0,2.0]| 32.24076584405401|
|   19| 74|  8|[19.0,74.0,8.0]|  9.74286069912749|
+-----+---+---+---------------+------------------+



### **Câu hỏi 2**: Train/test split

Tiếp tục với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

Giữa $80\%$ dữ liệu để làm training set, $20\%$ còn lại để làm test set, thế giá trị của `seed` là MSSV của bạn.

In [132]:
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed = 20133065)
print(f"Số phần tử trong training set: {trainDF.cache().count()}")
print(f"Số phần tử trong test set: {testDF.cache().count()}")

Số phần tử trong training set: 397
Số phần tử trong test set: 109


#### a. Thực hiện lại các bước tương tự như hương dẫn ở trên 

i. Huấn luyện mô hình trên training set

In [133]:
# Write your code here
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed=20133065) 

lr = LinearRegression(labelCol="medv", featuresCol="newFeatures")
lrModel = lr.fit(trainDF)

ii. In kết quả của mô hình (các hệ số (`Coefficients`, `Intercept`), `RMSE score`, và $R^2$ `score`) trên training set

In [134]:
# Write your code here
print("Coefficients: {0:.3f}, {1:.3f}, {2:.3f}".format(*lrModel.coefficients)) 
print("Intercept: {0:.3f}".format(lrModel.intercept))
print("R-squared score: {0:.3f}".format(lrModel.summary.r2))

# Write your code here
lrModelSummary = lrModel.summary

print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

Coefficients: -0.706, -0.097, -1.408
Intercept: 42.080
R-squared score: 0.301
MAE score: 5.304290311363103
RMSE score: 7.413062201507511
Coefficient Standard Errors: [0.08256773124569486, 0.020808455699436276, 0.303412578990576, 2.5303820412657405]
T Values: [-8.550991982037688, -4.6542272115174645, -4.640201821615346, 16.630091552018097]
P Values: [2.220446049250313e-16, 4.4530526539521276e-06, 4.749335799081322e-06, 0.0]


b. Đánh giá mô hình trên test set (in ra `RMSE score`)

In [135]:
# Write your code here
predictionDF = lrModel.transform(testDF)
rmse = predictionDF.rdd.map(lambda r: (r.medv - r.prediction)**2).mean()**0.5
print("RMSE on test set:", rmse)

RMSE on test set: 8.97642822175228


### **Câu hỏi 3**: Regularization với Ridge, Lasso, và ElasticNet regression

Tiếp tục với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

Điều chỉnh các tham số `regParam` (or `lambda`, regularization parameter) và `elasticNetParam` (or `alpha`, the ElasticNet mixing parameter, in range [0, 1]. For `alpha = 0`, the penalty is an `L2 penalty`. For `alpha = 1`, it is an `L1 penalty`.) của `LinearRegression`. Xem thêm ở [đây](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html#pyspark.ml.regression.LinearRegression).

- `regParam = 0`: ordinary least squares (OLS)

- `regParam != 0, elasticNetParam = 0`: L2 (Ridge regression)

- `regParam != 0, elasticNetParam = 1`: L1 (Lasso regression)

- `regParam != 0, elasticNetParam != 0`: L2 + L1 (ElasticNet regression)

#### a. `regParam = 0`: ordinary least squares

Thiết lập `regParam = 0` để tạo mô hình Linear Regression.

In [136]:
# Write your code here
lr = LinearRegression(labelCol="medv", featuresCol="newFeatures", regParam=0)


#### b. `regParam != 0, elasticNetParam = 0`: L2 (Ridge Regression)

Chọn một giá trị khác 0 cho tham số `regParam` để tạo mô hình Ridge Regression.

In [137]:
# Write your code here
lr = LinearRegression(labelCol="medv", featuresCol="newFeatures", regParam=0.3)

#### c. `regParam != 0, elasticNetParam = 1`: L1 (Lasso regression)

Chọn một giá trị khác 0 cho tham số `regParam` và thiết lập `elasticNetParam = 1` để tạo mô hình Lasso Regression.

In [138]:
# Write your code here
lr = LinearRegression(labelCol="medv", featuresCol="newFeatures", 
                      regParam=0.3, elasticNetParam=1)

#### d. `regParam != 0, elasticNetParam != 0`: L2 + L1 (ElasticNet Regression)

Chọn hai giá trị khác 0 cho tham số `regParam` và `elasticNetParam` để tạo mô hình ElasticNet Regression.

In [139]:
# Write your code here
lr = LinearRegression(labelCol="medv", featuresCol="newFeatures",  
                      regParam=0.3, elasticNetParam=0.5)

#### e. Nhận xét kết quả của các mô hình ở a, b, c, d.

- OLS có sự phù hợp tốt nhất (R-squared cao nhất) nhưng có thể phù hợp quá mức (overfitting). Các hệ số có thể không ổn định.

- Ridge regression thu hẹp các hệ số một chút so với OLS. Có thể giảm thiểu hiện tượng quá phù hợp.

- Lasso thu nhỏ một số hệ số về 0, dẫn đến lựa chọn tính năng. Có thể không phù hợp.(underfitting).

- ElasticNet cân bằng giữa ridge và lasso. Thu hẹp lại các hệ số và lựa chọn đặc tính.

### **Câu hỏi 4**: Tạo ML pipeline và đánh giá các mô hình dùng cross validation

![cross-validation-model-selection](./image/cross_val.png)

**Tham khảo**
1. ML Pipeline: https://spark.apache.org/docs/latest/ml-pipeline.html
2. ML Tuning: https://spark.apache.org/docs/latest/ml-tuning.html



**Lưu ý:** 

- Ở câu hỏi này, bạn cần sử dụng tất cả các thuộc tính khác `medv` làm input (`features`) và `medv` làm output (`label`) để tận dụng tối đa thông tin đã có nhất có thể.

- Bạn cần thiết lập các tham số `regParam` và `elasticNetParam` là một danh sách các giá trị để bao gồm các mô hình Linear Regression, Ridge Regression, Lasso Regression, và ElasticNet Regression. Mỗi tham số tối thiểu 5 giá trị khác nhau. Chú ý rằng: 

    - `regParam != 0` và `elasticNetParam = 0` ứng với Ridge Regression
    
    - `regParam != 0` và `elasticNetParam = 1` ứng với Lasso Regression
    
    - `regParam != 0` và `elasticNetParam != 0` ứng với ElasticNet Regression

In [146]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

featureCols = [col for col in bostonDF.columns if col != "medv"]

labelCol = "medv"

# Tạo VectorAssembler để tổng hợp các đặc trưng thành một vector
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

# Tạo StandardScaler để chuẩn hóa các đặc trưng
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

lr = LinearRegression(labelCol=labelCol)

# # Định nghĩa lưới siêu tham số
paramGrid = ParamGridBuilder() \
             .addGrid(lr.regParam, [0, 0.1, 0.5, 1, 5]) \
             .addGrid(lr.elasticNetParam, [0, 0.25, 0.5, 0.75, 1]) \
             .build()

# Tạo một luồng xử lý với các bước tổng hợp, chuẩn hóa và hồi quy tuyến tính
pipeline = Pipeline(stages=[assembler, scaler, lr])

# Tạo CrossValidator
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=RegressionEvaluator(labelCol=labelCol),
                    numFolds=5)

# Fit CrossValidator với dữ liệu

cvModel = cv.fit(bostonDF)
print(cvModel.avgMetrics)

[4.836735193574292, 4.836735193574292, 4.836735193574292, 4.836735193574292, 4.836735193574292, 4.830237395682609, 4.833419359129912, 4.8429905125768595, 4.858842985776664, 4.880664734779788, 4.839292857082275, 4.934230204326296, 5.026545690387849, 5.113077519265535, 5.201474048510017, 4.873374124875299, 5.060512439140187, 5.230054526696715, 5.318132153565073, 5.394920577848017, 5.239068773477873, 5.840761399347094, 6.402189920941031, 7.094414244004282, 7.866986834355373]


In [145]:
best_model = cvModel.bestModel

coefficients = best_model.stages[-1].coefficients
intercept = best_model.stages[-1].intercept

print("Coefficients: " + str(coefficients))
print("Intercept: " + str(intercept))

# Tính và in ra R-squared cho mô hình tốt nhất
predictions_best = cvModel.bestModel.transform(bostonDF)
evaluator = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_best)
print("R-squared: " + str(r2))


Coefficients: [-0.1031906901220606,0.04316811447289023,0.004052925895829607,2.750729094487936,-16.53255974895831,3.8694101483472267,-0.00042149591563641643,-1.4083274585771506,0.2663450605042258,-0.010445793259135331,-0.933110286382306,0.00928534810269963,-0.5151666620024974]
Intercept: 34.55457074284347
R-squared: 0.7403355590299989


#### Nhận xét, so sánh kết quả của các mô hình Linear Regression, Ridge Regression, Lasso Regression, và ElasticNet Regression

- Mô hình Hồi quy Tuyến tính thường cho kết quả phù hợp tốt nhất (R-squared cao nhất) nhưng có thể bị quá khớp (overfitting). Các hệ số có thể không ổn định.

- Mô hình Hồi quy Ridge thu hẹp các hệ số một chút so với Hồi quy Tuyến tính. Điều này có thể giúp giảm hiện tượng quá khớp.

- Mô hình Hồi quy Lasso thu nhỏ một số hệ số về 0, dẫn đến việc lựa chọn đặc trưng. Nhưng nó có thể bị không khớp (underfitting).

- Mô hình Hồi quy ElasticNet cân bằng giữa Ridge và Lasso. Nó vừa thu hẹp các hệ số vừa lựa chọn đặc trưng.


- Mô hình ElasticNet đạt R-squared tốt nhất là 0,74 trong khi vẫn giữ được tính điều chuẩn để tránh overfitting. Việc kiểm chứng chéo giúp tìm ra các giá trị tốt cho các tham số điều chuẩn alpha và lambda.

- So với Hồi quy Tuyến tính, Ridge, Lasso và ElasticNet áp dụng điều chuẩn giúp cải thiện khả năng khái quát hóa và ổn định của mô hình. Lasso làm giảm mạnh hệ số hơn nhưng ElasticNet cung cấp sự cân bằng tốt hơn.
- Kết luận : Hồi quy ElasticNet hoạt động tốt nhất cho tập dữ liệu này bằng cách tìm ra điểm cân bằng giữa giới hạn overfitting thông qua điều chuẩn trong khi vẫn duy trì độ phù hợp mô hình
